In [1]:
import requests
import json
import re
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def cleaner(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.findAll('p')

    text = ''
    p = re.compile('<.*?>')
    for paragraph in paragraphs:
        text += re.sub(p, '', str(paragraph))
    return text

In [3]:
def create_csv(label, data):
    df = pd.DataFrame(data=data, columns=['title', 'url', 'text'])
    df.to_csv(path_or_buf='./data/' + label + '.csv', index=False)

In [4]:
banks_en = ['absolutbank', 'akbars', 'alfabank', 'avangard',
            'bcs-bank', 'binbankcreditcard', 'binbank', 'bnpparibaseast',
            'bspb', 'citibank', 'crediteuropebank', 'deltacredit',
            'fk_otkritie', 'gazprombank', 'homecreditbank', 'jugra',
            'locko-bank', 'minbank', 'mkb', 'modulbank',
            'mosoblbank', 'mts-bank', 'otpbank', 'pochtabank',
            'promsvyazbank', 'qiwibank', 'raiffeisen', 'rencredit',
            'rgsbank', 'rosbank', 'roscap', 'rsb',
            'rshb', 'rusfinancebank', 'sberbank', 'skb-bank',
            'smpbank', 'sovcombank', 'tcs', 'touchbank',
            'transcapitalbank', 'trust', 'ubrr', 'unicreditbank',
            'uralsib', 'v-express-bank', 'vozrozhdenie', 'vtb24',
            'vtb', 'yandexdengi']
banks_ru = ['абсолют+банк', 'ак+барс+банк', 'альфа+банк', 'авангард+банк',
            'бкс+банк', 'бинбанк+кредит+кард', 'бинбанк', 'бнп+париба+восток',
            'банк+санкт+петербург', 'ситибанк', 'кредит+европа+банк', 'дельта+кредит',
            'фк+открытие', 'газпромбанк', 'хоум+кредит+банк', 'югра',
            'локо+банк', 'московский+индустриальный+банк', 'московский+кредитный+банк', 'модульбанк',
            'мособлбанк', 'мтс+банк', 'отп+банк', 'почта+банк',
            'промсвязьбанк', 'киви+банк', 'райффайзен', 'ренессанс+кредит',
            'росгосстрах+банк', 'росбанк', 'российский+капитал', 'русский+стандарт+банк',
            'россельхозбанк', 'русфинанс+банк', 'сбербанк', 'скб+банк',
            'смп+банк', 'совкомбанк', 'тинькофф', 'touch+bank',
            'транскапиталбанк', 'траст+банк', 'уральский+банк+реконструкции+и+развития', 'юникредит+банк',
            'уралсиб', 'восточный+экспресс+банк', 'возрождение+банк', 'втб24',
            'втб', 'яндекс+деньги']
unused_banks_ru = ['бинбанк+кредит+кард', 'российский+капитал']

In [5]:
url = None
response = None
prefix = 'https://lenta.ru/search/v2/process?from='
suffix = '0&size=10&sort=2&title_only=0&domain=1&modified,format=yyyy-MM-dd&bloc=4&query='

max_news = 20
for i in range(len(banks_en)):
    if (banks_ru[i] not in unused_banks_ru):
        bank_news = []
        continue_parsing = True
        index = 0
        while continue_parsing:
            if index == 0:
                url = prefix + suffix + banks_ru[i]
            else:
                url = prefix + str(index) + suffix + banks_ru[i]
            index += 1

            response = requests.get(url)
            if response.ok and len(response.json()['matches']) != 0:
                for news in response.json()['matches']:
                    bank_news.append({'title': news['title'], 'url': news['url'], 'text': cleaner(news['url'])})
                    if len(bank_news) == max_news:
                        continue_parsing = False
                        break
            else:
                continue_parsing = False

        create_csv(banks_en[i], bank_news)